<h4><center><center>WEBSCRAPING DE LAS FESTIVIDADES POR MES Y LUGAR EN CADA DEPARTAMENTO DEL PERÚ</center></h4>
<div align="center">
  <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSUjgS_8MwtXcTzBUO0KniN9nTMIpOMHbmaHw&s" alt="Alojamiento en Airbnb" width="300"/>
</div>

In [ ]:
#!pip install beautifulsoup4
#!pip install requests
#!pip install tqdm
#!pip install pandas

import bs4 as bs # BeautifulSoup para analizar el HTML
import requests # Para hacer peticiones HTTP a páginas web
from tqdm import tqdm # Para mostrar barra de progreso del código
import pandas as pd 

In [ ]:
# Lista de departamentos del Perú en formato URL (minúsculas y guiones donde corresponde)
departamentos = [
    "amazonas", "ancash", "apurimac", "arequipa", "ayacucho", "cajamarca",
    "cusco", "huancavelica", "huanuco", "ica", "junin", "la-libertad",
    "lambayeque", "lima", "loreto", "madre-de-dios", "moquegua", "pasco",
    "piura", "puno", "san-martin", "tacna", "tumbes", "ucayali"
]

# Listas vacías para almacenar los datos

Fechas = []
Lugar = []
Festividades = []
Departamento = []


# Recorrer cada departamento para extraer las festividades
for dpto in tqdm(departamentos):
    try:
        # Construir la URL de cada departamento
        url = f'https://www.serperuano.com/turismo/turismo-en-{dpto}/calendario-festivo-en-{dpto}/'
        # Realizar solicitud HTTP
        resp = requests.get(url)
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        # Buscar la tabla de festividades por su clase CSS
        table = soup.find('table', {'class': 'stcelda02'})

        if table:
            # Recorrer las filas de la tabla, omitiendo el encabezado
            for row in table.findAll('tr')[1:]:
                cols = row.findAll('td')
                # Verificar si hay al menos 3 columnas (fecha, lugar, festividad)
                if len(cols) >= 3:
                    Fechas.append(cols[0].text.strip())
                    Lugar.append(cols[1].text.strip())
                    Festividades.append(cols[2].text.strip())
                     # Formatear nombre del departamento (eliminar guiones y capitalizar)
                    Departamento.append(dpto.replace("-", " ").title())

        else:
            print(f" {dpto.title()}: No se encontró tabla")

    except Exception as e:
        # Captura de errores por cada departamento (para evitar que se detenga el script completo)
        print(f" {dpto.title()}: Error → {e}")

# Crear DataFrame
df = pd.DataFrame(list(zip(Departamento, Fechas, Lugar, Festividades)),
                  columns=['Departamento', 'Fechas', 'Lugar', 'Festividades'])

# Reiniciar índice
df = df.reset_index(drop=True)

# Mostrar los primeros registros
print(f"\n Total de festividades extraídas: {len(df)}")
df.head()


  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:12<00:00,  1.88it/s]


 Total de festividades extraídas: 777


,Departamento,Fechas,Lugar,Festividades
0,Amazonas,Febrero (Movible),Todo el departamento,Carnavales.
1,Amazonas,1-6 de mayo,Todo el departamento,Fiesta de las Cruces.
2,Amazonas,24 de junio,Bongará,Fiesta de San Juan.
3,Amazonas,25-29 de junio,Todo el departamento,Festividad de San Pedro y San Pablo.
4,Amazonas,Movible,Chachapoyas,Semana Turística.


In [ ]:
df.to_csv("festividades_departamentos_peru.csv", index=False, encoding="utf-8-sig")
